In [ ]:
import ast
import os
from collections import defaultdict


def get_python_files(package_dir):
    """
    Recursively find all Python files in the given directory.
    """
    python_files = []
    for root, _, files in os.walk(package_dir):
        for file in files:
            if file.endswith(".py"):
                python_files.append(os.path.join(root, file))
    return python_files


def extract_structure_from_file(file_path):
    """
    Parse a Python file using AST and extract classes, functions, and imports.
    """
    with open(file_path, "r", encoding="utf-8") as file:
        tree = ast.parse(file.read(), filename=file_path)

    classes = []
    functions = []
    imports = []

    for node in ast.walk(tree):
        if isinstance(node, ast.ClassDef):
            classes.append(node.name)
        elif isinstance(node, ast.FunctionDef):
            functions.append(node.name)
        elif isinstance(node, (ast.Import, ast.ImportFrom)):
            for alias in node.names:
                imports.append(alias.name)

    return classes, functions, imports


def analyze_package_structure(package_dir):
    """
    Analyze the package by traversing its files and extracting structure.
    """
    structure = defaultdict(lambda: {"classes": [], "functions": [], "imports": []})

    python_files = get_python_files(package_dir)
    for file in python_files:
        classes, functions, imports = extract_structure_from_file(file)
        structure[file]["classes"].extend(classes)
        structure[file]["functions"].extend(functions)
        structure[file]["imports"].extend(imports)

    return structure


def print_structure(structure):
    """
    Print the analyzed structure in a readable format.
    """
    for file, details in structure.items():
        print(f"\nFile: {file}")
        print("  Classes:")
        for cls in details["classes"]:
            print(f"    - {cls}")
        print("  Functions:")
        for func in details["functions"]:
            print(f"    - {func}")
        print("  Imports:")
        for imp in details["imports"]:
            print(f"    - {imp}")

In [ ]:
import graphviz


def visualize_structure(structure):
    dot = graphviz.Digraph(comment="Package Structure")

    for file, details in structure.items():
        file_node = os.path.basename(file)
        dot.node(file_node, file_node)

        # Add classes and functions as subnodes
        for cls in details["classes"]:
            dot.node(cls, cls)
            dot.edge(file_node, cls)

        for func in details["functions"]:
            dot.node(func, func)
            dot.edge(file_node, func)

    # Save without trying to open automatically
    output_path = dot.render("package_structure", format="pdf")
    print(f"File saved at: {output_path}")

In [ ]:
# Example usage
package_dir = "/project/Deep-Clustering/src"  # Replace with your package path
structure = analyze_package_structure(package_dir)
print_structure(structure)

In [ ]:
# Example usage
visualize_structure(structure)